In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset_real/gianpaolo/step_0.10/force/forward_accelerations.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(55, 56, 57, 58), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  10322


In [6]:
motion.shape

(10322, 59)

## Height-Force plots

In [7]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [8]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [9]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [10]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [11]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [12]:
np.round(0.2000, 1)

0.2

In [13]:
def create_acceleration_com_dataset(dataset, footsteps, motion=None, debug=False):
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 1:
            # Compute time difference between
            # current and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            # Round velocity array
            dataset[footsteps[idx-2], 1:4] = np.around(dataset[footsteps[idx-2], 1:4], decimals=1)
            dataset[footsteps[idx-1], 1:4] = np.around(dataset[footsteps[idx-1], 1:4], decimals=1)
            
            # Skip if actual velocity is 0.0
            if dataset[footsteps[idx-1], 1] == 0.0:
                    continue
            
            # Skip if accelerating from 0.0 to a velocity higher than 0.1
            if dataset[footsteps[idx-2], 1] == 0.0 and dataset[footsteps[idx-1], 1] != 0.1:
                continue
            
            # Skip if previous two footstep was from 0.0
            if dataset[footsteps[idx-2], 1] != 0.0 and dataset[footsteps[idx-1], 1] != 0.1 and dataset[footsteps[idx-3], 1] == 0.0:
                continue
            
            if time_difference < 0.7:
                lf_rh_moving = contacts[footsteps[idx], 0]
                rf_lh_moving = contacts[footsteps[idx], 1]
                
                if lf_rh_moving == rf_lh_moving:
                    print("Invalid")
                    continue

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 48:52])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 48:52])

                # Odom base vectors
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]

                # Rotate odometry estimation
                rotated_prev_base = prev_base @ R_prev
                rotated_curr_base = curr_base @ R_prev
                base_displacement = rotated_curr_base - rotated_prev_base
                
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 48:52])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 48:52])
                yaw = curr_yaw - prev_yaw
                
                if abs(yaw) > 0.05:
                    continue
                
                # Handle angle wrap
                if np.sign(prev_yaw) != np.sign(curr_yaw):
                    #print("Angle wrap detected.")
                    continue
                
                inputs.append(dataset[footsteps[idx-2], 1:4].tolist() +
                              dataset[footsteps[idx-1], 1:16].tolist() +
                              dataset[footsteps[idx-1], 19:23].tolist() +
                              dataset[footsteps[idx-1], 39:42].tolist() +
                              [lf_rh_moving, rf_lh_moving])
                
                # Labels
                labels.append([base_displacement[0], base_displacement[1], yaw])
                
    return np.array(inputs, dtype=object), np.array(labels, dtype=object)

# Stack datasets
X_motion_CoM_acceleration, Y_motion_CoM_acceleration = create_acceleration_com_dataset(motion, [x for x in range(len(motion))])
print(X_motion_CoM_acceleration.shape)
print(Y_motion_CoM_acceleration.shape)

(5387, 27)
(5387, 3)


In [14]:
%store X_motion_CoM_acceleration
%store Y_motion_CoM_acceleration

Stored 'X_motion_CoM_acceleration' (ndarray)
Stored 'Y_motion_CoM_acceleration' (ndarray)


In [15]:
min(Y_motion_CoM_acceleration[:, 0])

-0.005667350607364341

In [16]:
dic = dict()
for x in range(len(X_motion_CoM_acceleration)):
    key = str(round(X_motion_CoM_acceleration[x, 0], 1)) + str(round(X_motion_CoM_acceleration[x, 3], 1))
    if key in dic:
        dic[key][0].append(Y_motion_CoM_acceleration[x, 0])
        dic[key][1].append(X_motion_CoM_acceleration[x, -6])
    else:
        dic[key] = [[Y_motion_CoM_acceleration[x, 0]], [X_motion_CoM_acceleration[x, -6]]]

In [17]:
dic.keys()

dict_keys(['0.10.1', '0.00.1', '0.10.2', '0.20.2', '0.20.1', '0.10.3', '0.30.3', '0.30.1', '0.10.4', '0.40.4', '0.40.1', '0.10.5', '0.50.5', '0.50.1', '0.10.6', '0.60.6', '0.60.1', '0.10.7', '0.70.7', '0.70.1', '0.10.8', '0.80.8', '0.80.1', '0.10.9', '0.90.9', '0.90.1', '0.90.2', '0.20.3', '0.30.2', '0.20.4', '0.40.2', '0.20.5', '0.50.2', '0.20.6', '0.60.2', '0.20.7', '0.70.2', '0.20.8', '0.80.2', '0.20.9', '0.90.3', '0.30.4', '0.40.3', '0.30.5', '0.50.3', '0.30.6', '0.60.3', '0.30.7', '0.70.3', '0.30.8', '0.80.3', '0.30.9', '0.90.4', '0.40.5', '0.50.4', '0.40.6', '0.60.4', '0.40.7', '0.70.4', '0.40.8', '0.80.4', '0.40.9', '0.90.5', '0.50.6', '0.60.5', '0.50.7', '0.70.5', '0.50.8', '0.80.5', '0.50.9', '0.90.6', '0.60.7', '0.70.6', '0.60.8', '0.80.6', '0.60.9', '0.70.8', '0.80.7', '0.70.9', '0.90.7', '0.90.8', '0.80.9'])

In [18]:
#for x in range(len(dic["0.00.1"][0])):
#    print(dic["0.00.1"][0][x], dic["0.00.1"][1][x])

In [19]:
for key in dic.keys():
    print(f"Prev: {key[:3]}. Actual: {key[3:]}. \n\nDisp: {dic[key]}\n\n.")

Prev: 0.1. Actual: 0.1. 

Disp: [[0.02672861137026078, 0.030424394865291193, 0.024686803331151586, 0.03458490642286721, 0.031153428067646427, 0.030272511068051955, 0.03002168696438412, 0.03912789710850395, 0.033397712186359785, 0.03005440532816306, 0.029670455688083908, 0.02702554598777529, 0.03098520778543068, 0.02851791060291775, 0.03459191826547847, 0.028935245142902688, 0.03482120049525328, 0.02855735952677496, 0.032986857653638735, 0.030710286278534893, 0.035803087376217846, 0.02882066355599333, 0.030530377348600535, 0.032566362109403224, 0.03192295929007405, 0.028675216317591623, 0.031208231824649824, 0.030040830772571248, 0.03337473666529389, 0.03317084222912792, 0.028645079059450795, 0.033293664394378863, 0.02652780116243414, 0.02858937144074858, 0.03337247312811753, 0.03257369040727731, 0.030287062185073266, 0.015265436916012343, 0.033653806721751556, 0.03708028035939259, 0.030004344340032407, 0.02853522126207908, 0.03127106369710431, 0.01773708048039957, 0.03368180017770106, 

In [20]:
for key in dic.keys():
    print(f"Prev: {key[:3]}. Actual: {key[3:]}. Mean: {np.mean(dic[key][0])}. Std: {np.std(dic[key][0])}.")

Prev: 0.1. Actual: 0.1. Mean: 0.023851271324200448. Std: 0.01723455502503691.
Prev: 0.0. Actual: 0.1. Mean: 0.01461489186630901. Std: 0.009481655643851969.
Prev: 0.1. Actual: 0.2. Mean: 0.0437757346666667. Std: 0.007051518371534586.
Prev: 0.2. Actual: 0.2. Mean: 0.053519917823908816. Std: 0.01844391016908682.
Prev: 0.2. Actual: 0.1. Mean: 0.03853271006336024. Std: 0.005033296100193714.
Prev: 0.1. Actual: 0.3. Mean: 0.052977044865944733. Std: 0.018433891965793783.
Prev: 0.3. Actual: 0.3. Mean: 0.082353652484847. Std: 0.015311040136866462.
Prev: 0.3. Actual: 0.1. Mean: 0.0544786799553902. Std: 0.01282635751470908.
Prev: 0.1. Actual: 0.4. Mean: 0.07207660531146984. Std: 0.020238232017675977.
Prev: 0.4. Actual: 0.4. Mean: 0.11318657791089519. Std: 0.01547219819816176.
Prev: 0.4. Actual: 0.1. Mean: 0.07156082203361633. Std: 0.02373513434665735.
Prev: 0.1. Actual: 0.5. Mean: 0.08862070372299709. Std: 0.016983368950338768.
Prev: 0.5. Actual: 0.5. Mean: 0.145296792971185. Std: 0.01708328467249

In [21]:
#for x in range(400):
#    print(f"Prev Velocity: {X_motion_CoM_acceleration[x, -5:-2]}. Curr Velocity: {X_motion_CoM_acceleration[x, :3]}. Displacement: {Y_motion_CoM_acceleration[x, :]}.")